In [1]:
from srgan import SRGAN

Using TensorFlow backend.


In [2]:
from generator import ImageDataGenerator

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3419952138726437928,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13399335371175686930
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3425600492135811169
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5426053120
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14003611503266326832
 physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [5]:
vgg_path = 'vgg16_notop.hdf5'
# input_shape=(180, 320, 3)
# tgt_shape=(720, 1280, 3)

input_shape=(90, 160, 3)
tgt_shape=(360, 640, 3)

batch_size = 4
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
srgan.vgg_combined.get_weights()[0]

array([[[[ 0.08606051, -0.05234882,  0.04425201, ...,  0.01520357,
          -0.06748323, -0.05096979],
         [-0.0175508 ,  0.06030709,  0.02142777, ..., -0.02682538,
          -0.04096983,  0.03751595],
         [ 0.07221328,  0.06514113,  0.05326428, ..., -0.03666088,
          -0.04798609, -0.0876399 ]],

        [[ 0.08398595, -0.01195907,  0.04728703, ..., -0.07510273,
          -0.06248482, -0.01458228],
         [-0.0878499 ,  0.0753947 ,  0.03978202, ...,  0.08644071,
           0.05273072,  0.09213831],
         [ 0.09530979,  0.02884731,  0.05033267, ..., -0.00872453,
           0.08437349,  0.03664097]],

        [[ 0.09305282,  0.01674286,  0.03677106, ..., -0.06562889,
          -0.09447113, -0.0660251 ],
         [ 0.04388586,  0.01027964, -0.09475012, ..., -0.09910075,
           0.03042254, -0.08337608],
         [ 0.06772293,  0.01252749,  0.06844465, ...,  0.02299758,
          -0.01321091, -0.09843312]]],


       [[[ 0.09845194, -0.06643423, -0.07448137, ...,  0

In [7]:
input_dir = 'data/low/'
tgt_dir = 'data/high/'
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)

In [8]:
for e in range(100):
    print('epoch', e)
    data = gen.flow_from_directory().__next__()
    X_low = data[0]
    X_high = data[1]
    srgan.train(X_low, X_high)

epoch 0
d_loss 0.7055599689483643
epoch 1


FailedPreconditionError:  Error while reading resource variable _AnonymousVar91 from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/_AnonymousVar91/N10tensorflow3VarE does not exist.
	 [[node mul_43/ReadVariableOp (defined at /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_3772]

Function call stack:
keras_scratch_graph


In [10]:
srgan.vgg_combined.get_weights()[0]

array([[[[-0.06508176,  0.04992946, -0.07407328, ...,  0.06168107,
           0.01740799, -0.05072806],
         [-0.08460093, -0.07468732,  0.02369576, ..., -0.00293252,
          -0.06999168, -0.05292167],
         [-0.01016954, -0.03835202,  0.04949394, ...,  0.00383615,
           0.06248318, -0.10004897]],

        [[ 0.09925587,  0.09039931,  0.06024946, ..., -0.06724724,
           0.08268094,  0.01080219],
         [ 0.0634036 ,  0.08448839,  0.06680512, ..., -0.09816158,
          -0.04687847, -0.0011559 ],
         [ 0.03042686,  0.08640687,  0.04218038, ...,  0.09552835,
          -0.07627428, -0.0584987 ]],

        [[ 0.03262629,  0.00935973,  0.00226108, ...,  0.08369106,
           0.03963399, -0.02914942],
         [ 0.06770909, -0.03912432,  0.05862933, ...,  0.05250409,
          -0.08317128,  0.08657627],
         [-0.03766403,  0.0183902 , -0.04600696, ...,  0.0258602 ,
           0.03345634,  0.03690933]]],


       [[[-0.0914927 , -0.04935327,  0.09645455, ...,  0